In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
import os

from experiment import *

# Load data ** Chose your path before get stated **

In [2]:
path = os.getcwd()
rootDir, _ = path.split('ECG-Arritmia-Paper1')
path = 'ECG-Arritmia-Paper1/Database_extracted/Dataset_with_features'
filename = 'MIT-BIH__DS2_3classes__HOS.csv'
dataset_DS2 = pd.read_csv(os.path.join(rootDir, path, filename));

filename = 'MIT-BIH__DS1_3classes__HOS.csv'
dataset_DS1 = pd.read_csv(os.path.join(rootDir, path, filename));

In [3]:
dataset_DS1.head()

,Var1,Var2,Var3,Var4,Var5
0,2.466699,8.076415,0.130942,0.365797,0
1,2.388250,7.647216,0.118878,0.351855,0
2,2.270058,7.048029,0.147898,0.403758,0
3,2.235512,6.838190,0.157496,0.395403,0
4,2.269395,6.948087,0.165563,0.403978,0


In [4]:
dataset_DS1.shape

(50969, 5)

# Data preprocessing

In [5]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

In this step we are going standardize our dataset. The pipeline.pkl will be loaded, and it contains the weights of normalization onf DS1 set.

In [6]:
data_std_DS1 = std_scaler.fit_transform(dataset_DS1.values[:,:-1])
data_label_DS1 = dataset_DS1.values[:,-1]

data_std_DS2 = std_scaler.transform(dataset_DS2.values[:,:-1])
data_label_DS2 = dataset_DS2.values[:,-1]

# Load classifiers

In [8]:
random_search = joblib.load('./Models/classifiers.pkl')
random_search

{'MLP': RandomizedSearchCV(cv=8, error_score='raise',
           estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=(100,), learning_rate='adaptive',
        learning_rate_init=0.0005, max_iter=1300, momentum=0.9,
        nesterovs_momentum=True, power_t=0.5, random_state=None,
        shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
        verbose=False, warm_start=False),
           fit_params=None, iid=True, n_iter=20, n_jobs=10,
           param_distributions={'hidden_layer_sizes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, ...81, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000]},
      

In [11]:
from sklearn.externals import joblib

random_search = joblib.load('./Models/classifiers.pkl')
mlp_clf = random_search['MLP'].best_estimator_
svm_clf = random_search['SVM-RBF'].best_estimator_

from sklearn.naive_bayes import GaussianNB

bayes_clf = GaussianNB()

classifiers = {'MLP-238': mlp_clf, 'SVM-RBF': svm_clf, 'Naive-Bayes': bayes_clf}

# TRAIN on DS1 and **TEST** on DS2 set

In [12]:
clf_outputs = {'true': dict((k, {}) for k in classifiers.keys()), 
               'pred': dict((k, {}) for k in classifiers.keys())}
results = dict((k, {}) for k in classifiers.keys())

for clf in classifiers.keys():
    classifiers[clf].fit(data_std_DS1, data_label_DS1)
        
    clf_outputs['true'][clf][0] = data_label_DS2
    clf_outputs['pred'][clf][0] = classifiers[clf].predict(data_std_DS2)
    
    print(clf) 

MLP-238
SVM-RBF
Naive-Bayes


# Save results to CSVs and figures

In [13]:
clf_outputs

{'true': {'MLP-238': {0: array([0., 0., 0., ..., 0., 0., 0.])},
  'SVM-RBF': {0: array([0., 0., 0., ..., 0., 0., 0.])},
  'Naive-Bayes': {0: array([0., 0., 0., ..., 0., 0., 0.])}},
 'pred': {'MLP-238': {0: array([0., 0., 0., ..., 0., 0., 0.])},
  'SVM-RBF': {0: array([0., 0., 0., ..., 0., 0., 0.])},
  'Naive-Bayes': {0: array([0., 0., 0., ..., 0., 0., 0.])}}}

In [14]:
results = results_clf(3, clf_outputs['true'], clf_outputs['pred'])

/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/HOS/Experiment_05_scikitLearn__allCLF_onDS2_allFeatures_3Classes/experiment.py:150: RuntimeWarning: invalid value encountered in double_scalars
  metrics_class[i,2] = TP / (TP + FP)


In [17]:
export_results(results, 'validation')

MLP-238
SVM-RBF
Naive-Bayes


/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/HOS/Experiment_05_scikitLearn__allCLF_onDS2_allFeatures_3Classes/experiment.py:286: RuntimeWarning: invalid value encountered in true_divide
  return np.around((confMat / np.sum(confMat,axis=1)[:,None])*100,2)


In [16]:
save_models(classifiers)
save_pipeline(std_scaler)